In [45]:
from pytket.circuit import Op 


base_u1 = Op.create(OpType.U1, [0.25])

In [46]:
from pytket.circuit import QControlBox

controlled_u1 = QControlBox(base_u1, n=2)

In [85]:
from pytket.pauli import PauliStabiliser
from pytket.pauli import Pauli

phase_stabilise_z = PauliStabiliser([Pauli.Z], 1)


In [91]:
from pytket import Circuit
from pytket import OpType
from pytket.circuit.display import render_circuit_jupyter

my_circuit = Circuit(3, 5)
# block 1
my_circuit.H(0).X(1).X(2)
# block 2
my_circuit.add_gate(controlled_u1, [0, 1, 2])
my_circuit.add_gate(controlled_u1, [0, 1, 2])
my_circuit.add_gate(controlled_u1, [0, 1, 2])
my_circuit.add_gate(controlled_u1, [0, 1, 2])

#block 3
my_circuit.H(0)
my_circuit.Measure(0, 3)
my_circuit.Reset(0)
my_circuit.H(0)
my_circuit.Rz(-0.5, 0, condition_bits=[0], condition_value=1)

#block 4
my_circuit.add_gate(controlled_u1, [0, 1, 2])
my_circuit.add_gate(controlled_u1, [0, 1, 2])

#block 5
my_circuit.H(0)
my_circuit.Measure(0, 3)
my_circuit.Reset(0)
my_circuit.H(0)

#block 6
my_circuit.Rz(-0.25, 0, condition_bits=[0], condition_value=1)
my_circuit.Rz(-0.5, 0, condition_bits=[0], condition_value=1)
my_circuit.Rz(-0.75, 0, condition_bits=[0], condition_value=1)

# my_circuit.measure_all()

my_circuit.name = "test"

render_circuit_jupyter(my_circuit)